In [15]:
import os
import librosa
import numpy as np
import pandas as pd


In [16]:
def extract_mfcc(signal, sr, n_mfcc=40):
    mfcc = librosa.feature.mfcc(y=signal, sr=sr, n_mfcc=n_mfcc)
    mfcc_mean = np.mean(mfcc, axis=1)
    mfcc_std = np.std(mfcc, axis=1)
    return np.concatenate((mfcc_mean, mfcc_std))

def extract_zcr(signal):
    zcr = librosa.feature.zero_crossing_rate(signal)
    return np.array([np.mean(zcr), np.std(zcr)])

def extract_teo(signal):
    teo = np.zeros(len(signal))
    teo[1:-1] = signal[1:-1]**2 - signal[:-2] * signal[2:]
    return np.array([np.mean(teo), np.std(teo)])

def extract_hnr(signal, sr):
    f0, voiced_flag, voiced_probs = librosa.pyin(signal, fmin=50, fmax=300)
    harmonics = f0[~np.isnan(f0)]
    if len(harmonics) > 0:
        hnr = 10 * np.log10(np.var(harmonics) / (np.var(signal) + 1e-6))
    else:
        hnr = 0.0  # default if pitch not found
    return np.array([hnr])

def feature_concatenator(filepath):
    signal, sr = librosa.load(filepath, sr=None)

    # Optional: trim silence
    signal, _ = librosa.effects.trim(signal)

    features = []
    features.extend(extract_mfcc(signal, sr))
    features.extend(extract_zcr(signal))
    features.extend(extract_teo(signal))
    features.extend(extract_hnr(signal, sr))
    
    return np.array(features)




In [17]:
dataset_path=r'D:\Projects\MoodMate\augmented_dataset'
all_features=[]
all_labels=[]
for emotion in  os.listdir(dataset_path):
    for audio in os.listdir(os.path.join(dataset_path,emotion)):
        file_path=os.path.join(dataset_path,emotion,audio)
        extracted_features=feature_concatenator(file_path)

        all_features.append(extracted_features)
        all_labels.append(emotion)
    print(emotion," Done !")

X=np.array(all_features)
y=np.array(all_labels)

df=pd.DataFrame(X)
df['label']=y

df.to_csv("final_extracted_features.csv",index=False)

print("FEATURE EXTRACTION COMPLETED")
        

C:\Users\sudha\AppData\Local\Temp\ipykernel_16412\837251297.py:20: RuntimeWarning: divide by zero encountered in log10
  hnr = 10 * np.log10(np.var(harmonics) / (np.var(signal) + 1e-6))


angry  Done !
disgust  Done !
fear  Done !
happy  Done !
neutral  Done !
sad  Done !
FEATURE EXTRACTION COMPLETED
